In [22]:
import os
import shutil
from pathlib import Path

# Extracting data for raw layer


In [24]:
downloads_path = str(Path.home() / "Downloads")
notebook_dir = os.getcwd()  
raw_folder = os.path.join(notebook_dir, "raw")  

os.makedirs(raw_folder, exist_ok=True)

files = os.listdir(downloads_path)

for file in files:
    if file.startswith("EmployeeEngagement"):
        shutil.copy(os.path.join(downloads_path, file), raw_folder)

print(f"CSV files have been copied to {raw_folder}")

CSV files have been copied to C:\Users\PurnimaKabadwal\Desktop\Data-Engineering-Project-main\raw


# Starting Preparation Layer

In [26]:
import pandas as pd

# Load the data into dataframes
df_engagements_raw  = pd.read_csv('./raw/EmployeeEngagement.engagements.csv')
df_feedbacks_raw = pd.read_csv('./raw/EmployeeEngagement.feedbacks.csv')
df_learningmaterial_raw = pd.read_csv('./raw/EmployeeEngagement.learningmaterials.csv')
df_modules_raw = pd.read_csv('./raw/EmployeeEngagement.modules.csv')
df_quizzes_raw = pd.read_csv('./raw/EmployeeEngagement.quizzes.csv')
df_users = pd.read_csv('./raw/EmployeeEngagement.users.csv')
df_discussion = pd.read_csv('./raw/EmployeeEngagement.discussions.csv')

# # Display schema information for each dataframe
# print("Schema of df_engagements:")
# print(df_engagements.info())  # Displays column names, non-null values, and data types
# print("\nSample data of df_engagements:")
# print(df_engagements.head())  # Displays first 5 rows for a quick preview

# print("\nSchema of df_feedbacks:")
# print(df_feedbacks.info())
# print("\nSample data of df_feedbacks:")
# print(df_feedbacks.head())

# print("\nSchema of df_learningmaterial:")
# print(df_learningmaterial.info())
# print("\nSample data of df_learningmaterial:")
# print(df_learningmaterial.head())

# print("\nSchema of df_modules:")
# print(df_modules.info())
# print("\nSample data of df_modules:")
# print(df_modules.head())

# print("\nSchema of df_quizzes:")
# print(df_quizzes.info())
# print("\nSample data of df_quizzes:")
# print(df_quizzes.head())

# print("\nSchema of df_userss:")
# print(df_users.info())

# print("\nSchema of df_discussion:")
# print(df_discussion.info())
# print(df_users.head())


In [27]:

# Cleaning engagements data
df_engagements_clean = df_engagements_raw.dropna(subset=['userId', 'learningMaterialCompletion[0].learningMaterialId'])
df_engagements_clean['quizScore'] = df_engagements_clean['quizScore'].fillna(0) 

# Cleaning feedbacks data
df_feedbacks_clean = df_feedbacks_raw.dropna(subset=['userId', 'learningMaterialId']) 
df_feedbacks_clean['rating'] = df_feedbacks_clean['rating'].fillna(df_feedbacks_clean['rating'].mean()) 

# Cleaning learning material data
df_learningmaterial_clean = df_learningmaterial_raw.drop_duplicates(subset=['_id'])  
df_learningmaterial_clean['title'] = df_learningmaterial_clean['title'].fillna('No Title')  

# Cleaning modules data
df_modules_clean = df_modules_raw.drop_duplicates(subset=['_id']) 
df_modules_clean['description'] = df_modules_clean['description'].fillna('No Description')

# Cleaning quizzes data
df_quizzes_clean = df_quizzes_raw.drop_duplicates(subset=['_id']) 
df_quizzes_clean['title'] = df_quizzes_clean['title'].fillna('No Title') 

# Dimension and Fact Tables (Prep Layer)

# User Dimension Table
dim_users = df_engagements_clean[['userId']].drop_duplicates().reset_index(drop=True)
dim_users['userKey'] = dim_users.index + 1

# Learning Material Dimension Table
dim_learning_material = df_learningmaterial_clean[['_id', 'title', 'description', 'duration', 'department']].drop_duplicates()
dim_learning_material.columns = ['learningMaterialId', 'title', 'description', 'duration', 'department']
dim_learning_material['learningMaterialKey'] = dim_learning_material.index + 1

# Module Dimension Table
dim_modules = df_modules_clean[['_id', 'title', 'description', 'learningMaterialId']].drop_duplicates()
dim_modules.columns = ['moduleId', 'title', 'description', 'learningMaterialId']
dim_modules['moduleKey'] = dim_modules.index + 1

# Quiz Dimension Table
dim_quizzes = df_quizzes_clean[['_id', 'title', 'learningMaterialId']].drop_duplicates()
dim_quizzes.columns = ['quizId', 'title', 'learningMaterialId']
dim_quizzes['quizKey'] = dim_quizzes.index + 1

# Fact Table for Engagements
fact_engagements = df_engagements_clean.merge(dim_users, on='userId', how='left')
fact_engagements = fact_engagements.merge(dim_learning_material, 
                                          left_on='learningMaterialCompletion[0].learningMaterialId',
                                          right_on='learningMaterialId', 
                                          how='left')
fact_engagements['engagementKey'] = fact_engagements.index + 1
fact_engagements = fact_engagements[['engagementKey', 'userKey', 'learningMaterialKey', 'quizScore', 'learningMaterialCompletion[0].completed']]

# Fact Table for Feedbacks
fact_feedbacks = df_feedbacks_clean.merge(dim_users, on='userId', how='left')
fact_feedbacks = fact_feedbacks.merge(dim_learning_material, on='learningMaterialId', how='left')
fact_feedbacks['feedbackKey'] = fact_feedbacks.index + 1
fact_feedbacks = fact_feedbacks[['feedbackKey', 'userKey', 'learningMaterialKey', 'feedback', 'rating']]


 

In [28]:
dataframes = {
    "fact_engagements.csv":fact_engagements,
    "dim_quizzes.csv": dim_quizzes,
    "fact_feedbacks.csv": fact_feedbacks,
    "dim_modules.csv" : dim_modules,
    "dim_learningmaterials.csv" :dim_learning_material,
    "dim_users.csv" :dim_users
}


prep = "prep"

for name, df in dataframes.items():
    print(f"Schema for {name}:")
    print(df.dtypes)
    print("\n")
if not os.path.exists(prep):
    os.makedirs(prep)

for filename, df in dataframes.items():
    file_path = os.path.join(prep, filename)
    df.to_csv(file_path, index=False) 
print(f"All DataFrames have been saved in the '{prep}' folder.")

Schema for fact_engagements.csv:
engagementKey                              int64
userKey                                    int64
learningMaterialKey                        int64
quizScore                                  int64
learningMaterialCompletion[0].completed     bool
dtype: object


Schema for dim_quizzes.csv:
quizId                object
title                 object
learningMaterialId    object
quizKey                int64
dtype: object


Schema for fact_feedbacks.csv:
feedbackKey              int64
userKey                float64
learningMaterialKey      int64
feedback                object
rating                   int64
dtype: object


Schema for dim_modules.csv:
moduleId              object
title                 object
description           object
learningMaterialId    object
moduleKey              int64
dtype: object


Schema for dim_learningmaterials.csv:
learningMaterialId     object
title                  object
description            object
duration                int

In [29]:
import pandas as pd

# Load the prepped/cleaned data from the previous layer
dim_users = pd.read_csv('./prep/dim_users.csv')
dim_learning_material = pd.read_csv('./prep/dim_learningmaterials.csv')
dim_modules = pd.read_csv('./prep/dim_modules.csv')
dim_quizzes = pd.read_csv('./prep/dim_quizzes.csv')
fact_engagements = pd.read_csv('./prep/fact_engagements.csv')
fact_feedbacks = pd.read_csv('./prep/fact_feedbacks.csv')

# Summary Fact Table for Engagements
# Group by learning material and calculate total, average, and other engagement metrics
fact_engagement_summary = fact_engagements.groupby('learningMaterialKey').agg(
    total_engagements=('engagementKey', 'count'),
    total_quiz_score=('quizScore', 'sum'),
    avg_quiz_score=('quizScore', 'mean'),
    total_completed=('learningMaterialCompletion[0].completed', 'sum')
).reset_index()


fact_engagement_summary = fact_engagement_summary.merge(dim_learning_material, on='learningMaterialKey', how='left')
fact_engagement_summary = fact_engagement_summary[['learningMaterialKey', 'title', 'department', 'total_engagements', 'total_quiz_score', 'avg_quiz_score', 'total_completed']]

# Summary Fact Table for Feedbacks
# Group by learning material and calculate average rating
fact_feedback_summary = fact_feedbacks.groupby('learningMaterialKey').agg(
    total_feedbacks=('feedbackKey', 'count'),
    avg_rating=('rating', 'mean')
).reset_index()

fact_feedback_summary = fact_feedback_summary.merge(dim_learning_material, on='learningMaterialKey', how='left')
fact_feedback_summary = fact_feedback_summary[['learningMaterialKey', 'title', 'department', 'total_feedbacks', 'avg_rating']]

# Star Schema by Joining Fact and Dimension Tables
# The star schema would link the fact tables to their corresponding dimension tables
fact_engagement_star = fact_engagements.merge(dim_users, on='userKey', how='left')
fact_engagement_star = fact_engagement_star.merge(dim_learning_material, on='learningMaterialKey', how='left')
fact_engagement_star = fact_engagement_star[['engagementKey', 'userKey', 'userId', 'learningMaterialKey', 'title', 'department', 'quizScore', 'learningMaterialCompletion[0].completed']]

fact_feedback_star = fact_feedbacks.merge(dim_users, on='userKey', how='left')
fact_feedback_star = fact_feedback_star.merge(dim_learning_material, on='learningMaterialKey', how='left')
fact_feedback_star = fact_feedback_star[['feedbackKey', 'userKey', 'userId', 'learningMaterialKey', 'title', 'department', 'feedback', 'rating']]


In [30]:
reorting_dataframes = {
    "fact_engagement_summary.csv":fact_engagement_summary,
    "fact_feedback_summary.csv":fact_feedback_summary,
    "fact_engagement_star.csv": fact_engagement_star,
    "fact_feedback_star.csv" : fact_feedback_star
}


reporting = "reporting"

for name, df in reorting_dataframes.items():
    print(f"Schema for {name}:")
    print(df.dtypes)
    print("\n")
if not os.path.exists(reporting):
    os.makedirs(reporting)

for filename, df in reorting_dataframes.items():
    file_path = os.path.join(reporting, filename)
    df.to_csv(file_path, index=False) 
print(f"All DataFrames have been saved in the '{reporting}' folder.")

Schema for fact_engagement_summary.csv:
learningMaterialKey      int64
title                   object
department              object
total_engagements        int64
total_quiz_score         int64
avg_quiz_score         float64
total_completed          int64
dtype: object


Schema for fact_feedback_summary.csv:
learningMaterialKey      int64
title                   object
department              object
total_feedbacks          int64
avg_rating             float64
dtype: object


Schema for fact_engagement_star.csv:
engagementKey                               int64
userKey                                     int64
userId                                     object
learningMaterialKey                         int64
title                                      object
department                                 object
quizScore                                   int64
learningMaterialCompletion[0].completed      bool
dtype: object


Schema for fact_feedback_star.csv:
feedbackKey              int64